In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# ML tools 
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from tensorflow.keras import Model
# import tensorflow.keras.applications.efficientnet as efn
from tensorflow.keras.applications import *
import os
from keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [ ]:
from keras.models import load_model
import cv2
denseNet_model = load_model('../input/densenet121-14-diseases-softmax/Densenet_model.h5')
inceptionv2_model = load_model('../input/inceptionv2-14-diseases-softmax/Inception_model.h5')
# mobilenet_model = load_model('')

In [ ]:
def DensepredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (256, 256))
    img = img.reshape((1,256,256,3))
    return denseNet_model.predict(img)

def InceptionpredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (256, 256))
    img = img.reshape((1,256,256,3))
    return inceptionv2_model.predict(img)

def mobilepredictClass(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img/255
    img = cv2.resize(img, (128, 128))
    img = img.reshape((1,128,128,3))
    return mobilenet_model.predict(img)

In [ ]:
disease_names = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural Thickening","Hernia"]

In [ ]:
def classnames(arr):
    for i in range(arr.size):
        print("Disease:", disease_names[i], " Probability:",arr[i]*100)
def meanf(arr1, arr2, arr3):
    for i in range(14):
        print("Disease:", disease_names[i], " Probability:",((arr1[i]+arr2[i]+0)/3) * 100)

In [ ]:
def soft_voting(path):
    x = DensepredictClass(path)
    y = InceptionpredictClass(path)
#     z = mobilepredictClass(path)
    print('\nSoft Voting ImageName:-',path)
    return meanf(x[0], y[0], 0) 

# Extracting Image Name

In [ ]:
DATASET_NAME = "nih-image-600x600-data"

In [ ]:
from kaggle_datasets import KaggleDatasets
GCS_DS_PATH = KaggleDatasets().get_gcs_path(DATASET_NAME)
GCS_DS_PATH

In [ ]:
df = pd.read_csv('../input/nih-dataframe/NIH_Dataframe.csv')
df.img_ind= df.img_ind.apply(lambda x: x.split('.')[0])
display(df.head(4))
print(df.shape)

In [ ]:
paths = GCS_DS_PATH + "/NIH_Images/" + df['img_ind'] + '.jpg'
#Get the multi-labels
label_cols = df.columns[:-1]
labels = df[label_cols].values

In [ ]:
(train_paths, valid_paths, 
  train_labels, valid_labels) = train_test_split(paths, labels, test_size=0.2, random_state=11)

In [ ]:
test = pd.read_csv('../input/data/Data_Entry_2017.csv')
test.head()
valid = valid_paths.reset_index()
valid.drop(['index'],inplace=True, axis = 1)
print(valid)
valid['img_ind']

In [ ]:
import cv2

In [ ]:
type((valid['img_ind'][0]).split('/')[-1])

In [ ]:
diseases = ["Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax","Consolidation","Edema","Emphysema","Fibrosis","Pleural_Thickening","Hernia"]


In [ ]:
#Pass Image Name to get labels
def actual_labels(img_name):
    df.loc[df['img_ind'] == img_name]
    print("Actual Labels:")
    for disease in diseases:
        if df.loc[df['img_ind'] == img_name][disease][1]:
            print(disease)

In [ ]:
actual_labels((valid['img_ind'][0]).split('/')[-1])

In [ ]:
pred = []
for i in range(2):
    img = valid['img_ind'][i]
    actual_labels((valid['img_ind'][i]).split('/')[-1])
    img = '../input/nih-image-600x600-data/NIH_Images/' + img.split('/')[-1]
    pred.append(soft_voting(img))
    
    


In [ ]:
print('AUC CKECK-UP per CLASS')

classes= df.columns[:-1]
for i, n in enumerate(classes):
  print(classes[i])
  print(i, roc_auc_score(valid_labels[:, i], pred[:, i]))
  print('---------')

# Checking Labels of Image

**Creating DataFrame for Images Labels**

In [ ]:
df = pd.read_csv('../input/nih-dataframe/NIH_Dataframe.csv')
df.img_ind= df.img_ind.apply(lambda x: x.split('.')[0])
df = df.reset_index()
df.drop(['index'],inplace=True, axis = 1)
df.set_index('img_ind')

In [ ]:
# tf.keras.backend.clear_session()

# from sklearn.metrics import roc_auc_score

# ensemble_model= tf.keras.models.load_model(name)
# pred = model.predict(dvalid, verbose=1)

print('AUC CKECK-UP per CLASS')

classes= df.columns[:-1]
for i, n in enumerate(classes):
  print(classes[i])
  print(i, roc_auc_score(valid_labels[:, i], pred[:, i]))
  print('---------')